# Gradient Boosting
(by Tevfik Aytekin)

In [72]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor


def kaggle_score(y_true,y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred));
#def kaggle_score(y_true,y_pred):
#    return np.sqrt(mean_squared_error(np.log(y_true), np.log(y_pred)));
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [48]:
house_train = pd.read_csv("../../datasets/house_prices/train.csv")


### DecisionTreeRegressor
Run DecisionTreeRegressor on House Prices.

In [49]:
X = house_train.loc[:,'MSSubClass':'SaleCondition']
y = house_train.loc[:,'SalePrice']
X = pd.get_dummies(X)
mae, kaggle, mape = [], [], []
for i in range(1,10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    X_train = X_train.fillna(X_train.mean())
    X_test = X_test.fillna(X_test.mean())

    regr = DecisionTreeRegressor()
    regr.fit(X_train, y_train)
    
    test_predictions = regr.predict(X_test)
    mae.append(mean_absolute_error(y_test, test_predictions))
    mape.append(mean_absolute_percentage_error(y_test, test_predictions))
    kaggle.append(kaggle_score(y_test, test_predictions))

print("Test MAE:", np.mean(mae))
print("Test MAPE:", np.mean(mape))
print("Test Kaggle:", np.mean(kaggle))


Test MAE: 25149.729071537287
Test MAPE: 14.535382715783701
Test Kaggle: 0.19832978682393862


### GradientBoostingRegressor
Run sklearn's GradientBoostingRegressor on House Prices dataset

In [50]:
X = house_train.loc[:,'MSSubClass':'SaleCondition']
y = house_train.loc[:,'SalePrice']
X = pd.get_dummies(X)
mae, kaggle, mape = [], [], []
for i in range(1,10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    X_train = X_train.fillna(X_train.mean())
    X_test = X_test.fillna(X_test.mean())

    regr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1)
    regr.fit(X_train, y_train)
    
    test_predictions = regr.predict(X_test)
    mae.append(mean_absolute_error(y_test, test_predictions))
    mape.append(mean_absolute_percentage_error(y_test, test_predictions))
    kaggle.append(kaggle_score(y_test, test_predictions))

print("Test MAE:", np.mean(mae))
print("Test MAPE:", np.mean(mape))
print("Test Kaggle:", np.mean(kaggle))


Test MAE: 15447.651721974144
Test MAPE: 9.045724745406021
Test Kaggle: 0.12663816676926054


### GradientBoostingRegressor from scratch
Let us write GradientBoostingRegressor from scratch. Note that "learning_rate" is also known as "shrinkage."

In [51]:
class MyGradientBoostingRegressor:
    
    def __init__(self, n_estimators = 100, learning_rate = 0.1, max_depth=3):
        self.models = []
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
    def calc_grads(self, model, X, y):    
        preds = self.learning_rate * model.predict(X)
        grads = y - preds
        return grads
    def predict(self, X):
        preds = np.zeros(X.shape[0])
        for m in self.models:
            preds += self.learning_rate * m.predict(X)
        return preds
        
    def fit(self, X, y):
        for i in range(self.n_estimators):
            model = DecisionTreeRegressor(max_depth=self.max_depth);
            if (i == 0):
                model.fit(X, y)
                grads = self.calc_grads(model, X, y)
            else:
                model.fit(X, grads)
                grads = self.calc_grads(model, X, grads)
            self.models.append(model)
            
            

### How learning_rate works

learning_rate shrinks the contribution of each tree. This is an important parameter which has significant effect on the error. You can try to set it to 1 and look at the result. Below is a simple illustration:

Suppose that we have only 3 models: M1, M2, and M3 <br>
y = 10 <br>
learning_rate = 1 (no shrinkage)


M1.predict = 8 <br>
grad = 10 - 8 = 2

fit M2 for y = 2 <br>
M2.predict = 1.5 <br>
grad = 2 - 1.5 = 0.5

fit M3 for y = 0.5


Prediction:

M1.predict(X) + M2.predict(X) + M3.predict(X)

-----------------------
3 models: M1, M2, and M3 <br>
y = 10<br>
learning_rate = 0.1 (with shrinkage)

M1.predict = 8 <br>
grad = 10 - 0.1 x 8 = 9.2

fit M2 for y = 9.2<br>
M2.predict = 7<br>
grad = 9.2 - 0.1 * 7 = 8.5

fit M3 for y = 8.5


Prediction:

0.1 x M1.predict(X) + 0.1 x M2.predict(X) + 0.1 x M3.predict(X) 

?? Should we multiply M3's prediction with 0.1?

**Question:** Why does shrinkage improve performance significantly?




### Example Run
Now let us run our version of GradientBoostingRegressor on the same dataset

In [66]:
from sklearn.metrics import mean_absolute_error
X = house_train.loc[:,'MSSubClass':'SaleCondition']
y = house_train.loc[:,'SalePrice']
X = pd.get_dummies(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

reg1 = DecisionTreeRegressor()
reg2 = GradientBoostingRegressor()
reg3 = MyGradientBoostingRegressor(learning_rate=1, max_depth=3)

reg1.fit(X_train, y_train)
reg2.fit(X_train, y_train)
reg3.fit(X_train, y_train)

print(mean_absolute_percentage_error(y_test, reg1.predict(X_test)))
print(mean_absolute_percentage_error(y_test, reg2.predict(X_test)))
print(mean_absolute_percentage_error(y_test, reg3.predict(X_test)))

15.307269085250777
11.149679196688858
16.151335711659147


### Gradient Boosting Classifier

In [ ]:
# Bank Marketing Dataset from
# https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

bank = pd.read_csv("../datasets/bank/bank-full.csv", delimiter = ";")
# print first 5 examples
bank.head(5)

In [ ]:
bank.y.value_counts()

In [ ]:
bank_majority = bank[bank.y=="no"]
bank_minority = bank[bank.y=="yes"]
 
# downsample
bank_majority_downsampled = resample(bank_majority, 
                                 replace=False,    
                                 n_samples=5289) 
 
bank_balanced = pd.concat([bank_minority, bank_majority_downsampled])
bank_balanced.y.value_counts()

### Logistic/Sigmoid function

In [ ]:
x = np.arange(-6, 6, 0.1)
s = 1 / (1 + np.exp(-x))
plt.plot(x, s)
plt.xlabel('x')
plt.ylabel('s(x)')
plt.suptitle("Logistic/Sigmoid function")
plt.show()

In [ ]:
class MyGradientBoostingClassifier:
    
    def cross_ent(self, y, p):
        return - y * np.log(p) - (1 - y) * np.log(1 - p)
    
    def diff(self, y, p):
        return y - p
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x)) 
    
    def __init__(self, n_estimators = 100, learning_rate = 0.1, max_depth=3):
        self.models = []
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        
    def calc_grads(self, model, X, y):    
        preds = self.learning_rate * model.predict(X)
        grads = self.diff(y, preds)
        return grads
    
    def predict(self, X):
        preds = np.zeros(X.shape[0])
        for m in self.models:
            preds += self.learning_rate * m.predict(X)
        return np.round(preds)
        
    def fit(self, X, y):
        for i in range(self.n_estimators):
            model = DecisionTreeRegressor(max_depth=2);
            if (i == 0):
                model.fit(X, y)
                grads = self.calc_grads(model, X, y)
            else:
                model.fit(X, grads)
                grads = self.calc_grads(model, X, grads)
            self.models.append(model)
            

In [ ]:
X = bank_balanced.loc[:,'age':'poutcome']
y = bank_balanced.loc[:,'y']
y = y.replace(["yes","no"],[1,0])
X = pd.get_dummies(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())
 
clf1 = DecisionTreeClassifier()
clf2 = GradientBoostingClassifier()
clf3 = MyGradientBoostingClassifier()

clf1.fit(X_train, y_train);
clf2.fit(X_train, y_train);
clf3.fit(X_train, y_train);

y_pred1 = clf1.predict(X_test)  
y_pred2 = clf2.predict(X_test) 
y_pred3 = clf3.predict(X_test) 

print(classification_report(y_test,y_pred1))
print(classification_report(y_test,y_pred2))
print(classification_report(y_test,y_pred3))

### Many Boosting Frameworks
Boosting is the most popular/successful method for tabular datasets, hence there are many boosting libraries. Some of them are given below.

In [75]:
import time

X = house_train.loc[:,'MSSubClass':'SaleCondition']
y = house_train.loc[:,'SalePrice']
X = pd.get_dummies(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())


models = [DecisionTreeRegressor(), GradientBoostingRegressor(), MyGradientBoostingRegressor(), 
          XGBRegressor(), LGBMRegressor(verbose= -1), CatBoostRegressor(verbose=False), 
          HistGradientBoostingRegressor(), RandomForestRegressor()]

for model in models:
    start_time = time.time()
    model.fit(X_train, y_train)
    end_time = time.time()
    elapsed_time = end_time-start_time
    mae = mean_absolute_percentage_error(y_test, model.predict(X_test))    
    print(np.round(mae,2),f"Elapsed time: {np.round(elapsed_time,2)} seconds")


15.03 Elapsed time: 0.07 seconds
9.55 Elapsed time: 1.11 seconds
9.56 Elapsed time: 3.27 seconds
9.88 Elapsed time: 0.83 seconds
9.45 Elapsed time: 0.15 seconds
8.46 Elapsed time: 3.71 seconds
9.21 Elapsed time: 1.55 seconds
10.22 Elapsed time: 3.68 seconds


Some libraries can handle missing values.

In [79]:

X = house_train.loc[:,'MSSubClass':'SaleCondition']
y = house_train.loc[:,'SalePrice']
X = pd.get_dummies(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

models = [DecisionTreeRegressor(), XGBRegressor(), LGBMRegressor(verbose= -1), 
          HistGradientBoostingRegressor(),CatBoostRegressor(verbose=False)]

for model in models:
    model.fit(X_train, y_train)
    print(mean_absolute_percentage_error(y_test, model.predict(X_test)))

25.815124828097503
10.320379532677274
9.341487041963365
9.742128691329663
8.66274472557568


### XGBoost Classification

In [ ]:
X = bank_balanced.loc[:,'age':'poutcome']
y = bank_balanced.loc[:,'y']
y = y.replace(["yes","no"],[1,0])
X = pd.get_dummies(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())
 
clf1 = DecisionTreeClassifier()
clf2 = GradientBoostingClassifier()
clf3 = MyGradientBoostingClassifier()
clf4 = xgb.XGBClassifier()

clf1.fit(X_train, y_train);
clf2.fit(X_train, y_train);
clf3.fit(X_train, y_train);
clf4.fit(X_train, y_train);

y_pred1 = clf1.predict(X_test)  
y_pred2 = clf2.predict(X_test) 
y_pred3 = clf3.predict(X_test)
y_pred4 = clf4.predict(X_test) 

print(classification_report(y_test,y_pred1))
print(classification_report(y_test,y_pred2))
print(classification_report(y_test,y_pred3))
print(classification_report(y_test,y_pred4))

In [ ]:
X = bank_balanced.loc[:,'age':'poutcome']
y = bank_balanced.loc[:,'y']
y = y.replace(["yes","no"],[1,0])
X = pd.get_dummies(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

clf4 = xgb.XGBClassifier()
clf4.fit(X_train, y_train);
y_pred4 = clf4.predict(X_test) 
print(classification_report(y_test,y_pred4))